# Introduction to AWS ParallelCluster

This is a shortened version. The creation of the ParallelCluster is hidden in pcluster-athena.py. 


In [ ]:
import boto3
import botocore
import json
import time
import os
import sys
import base64
import docker
import pandas as pd
import importlib
import project_path # path to helper methods
from lib import workshop
from botocore.exceptions import ClientError

sys.path.insert(0, '.')
import pcluster_athena
importlib.reload(pcluster_athena)


# ssh key for access the pcluster. this key is not needed  in this excercise, but useful if you need to ssh into the headnode of the pcluster
key_name = 'pcluster-athena-key'
keypair_saved_path = './'+key_name+'.pem'
# unique name of the pcluster
pcluster_name = 'myTestCluster'
# the slurm REST token is generated from the headnode and stored in Secrets Manager. This token is used in makeing REST API calls to the Slurm REST endpoint running on the headnode 
slurm_secret_name = "slurm_token_{}".format(pcluster_name)
# We only need one subnet for the pcluster, but two subnets are needed for RDS instance. If use existing VPC, we will use the default VPC, and the first subnet in default VPC
use_existing_vpc = True



## Create the parallel cluster
In this simpler version, we will run the cluster creation in the background. The process will take around 10 minutes. The two steps that take longer than the rest are the creation of a MySQL RDS instance, and the cluster itself. 

If you want to see how each step is done, please use the "pcluster-athena++" notebook in the same directory.
 

In [ ]:
# create the cluster - # You can rerun the rest of the notebook again with no harm. There are checks in place for existing resoources. 

my_bucket_name, db_name, slurm_secret_name, rds_secret_name = pcluster_athena.create_pcluster()



In [ ]:

pcluster_status = !pcluster status $pcluster_name

# get the second part of 'MasterPrivateIP: 172.16.2.92'
slurm_host = pcluster_status.grep('MasterPrivateIP').s.split()[1]

print(slurm_host)

### Integrate with Slurm REST API running on the head node

SLURM REST is currently running on the headnode, using jwt as the auth mechanism. 

First from the openapi endpoint /openapi/v3, we can exam the schema of the Slurm REST API. 


### Store the JWT token in Secrete Manager
JWT token can be created using the "scontrol token username=slurm" command on the head-node. To pass it securely to this notebook, we will first create a cron job on the headnode to retrieve the token, then save it in SecreteManager with a name "slurm_token". The default JWT token lifespan is 1800 seconds(30 mins). Run the follow script on the head-node as a cron job to update the token every 20 mins

The following steps are included in the post_install_script. You DO NOT need to run it. 
#### Step 1.  Add permission to the instance role for the head-node

```
{
    "Action": [ 
        "secretsmanager:DescribeSecret",
        "secretsmanager:CreateSecret",
        "secretsmanager:UpdateSecret"],
    "Resource": [
        "arn:aws:secretsmanager:us-east-1:<account-id>:secret:*"
    ],
    "Effect": "Allow",
    "Sid": "tokensecret"
}
```

**Skip step 2 and step 3 - They are now included in the post_install script**

#### Step 2. Create a script "token_refresher.sh" 
Assume we save the following script at /shared/token_refresher.sh 

``` token_refresher.sh
#!/bin/bash

REGION=us-east-1
export $(/opt/slurm/bin/scontrol token -u slurm)

aws secretsmanager describe-secret --secret-id slurm_token --region $REGION

if [ $? -eq 0 ]
then
 aws secretsmanager update-secret --secret-id slurm_token --secret-string "$SLURM_JWT" --region $REGION
else
 aws secretsmanager create-secret --name slurm_token --secret-string "$SLURM_JWT" --region $REGION
fi
```

#### Step 3. Add a file "slurm-token" in /etc/cron.d/

```/etc/cron.d/slurm-token
# Run the slurm token update every 20 minues 
SHELL=/bin/bash
PATH=/sbin:/bin:/usr/sbin:/usr/bin
MAILTO=root
*/20 * * * * root /shared/token_refresher.sh                                       
```

#### Step 4. Add permission to access SecretManager for this notebook

Don't forget to add secretsmanager:GetSecretValue permission to the sagemaker execution role that runs this notebook

In [ ]:
import boto3
import base64
import time
from botocore.exceptions import ClientError
from IPython.display import HTML, display

slurm_user = 'slurm'

def display_table(data):
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td><h4>%s</h4><td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

###
# Retrieve the slurm_token from the SecretManager
#
def get_secret():
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(SecretId=slurm_secret_name)
    except ClientError as e:
        print("Error", e)
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
            return secret
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            return decoded_binary_secret

###
# Retrieve the token and inject into the header for JWT auth
#
def update_header_token():
    token = get_secret()
    post_headers = {'X-SLURM-USER-NAME':slurm_user, 'X-SLURM-USER-TOKEN': token, 'Content-type': 'application/json', 'Accept': 'application/json'}
    get_headers = {'X-SLURM-USER-NAME':slurm_user, 'X-SLURM-USER-TOKEN': token, 'Content-type': 'application/x-www-form-urlencoded', 'Accept': 'application/json'}
    return [post_headers, get_headers]

###
# Convert response into json
#
def convert_response(resp):
    resp_str = resp.content.decode('utf-8')
    return json.loads(resp_str)



###
# Print a json array in table format
# input: headers [json attribute name, ... ]
# input: a - array of json objects
def print_table_from_json_array(headers, a):
    # add headers as the first row.
    t = [headers]
    for item in a:
        result = []
        for h in headers:
            result.append(item[h])
        t.append(result)
    display_table(t)

def print_table_from_dict(headers, d):
    result = list()
    for k,v in d.items():
        result.append(v)
    print_table_from_json_array(headers, result)
        

### 
# wrapper for get
#
def get_response_as_json(base_url):
    _, get_headers = update_header_token()
    resp = requests.get(base_url, headers=get_headers)
    if resp.status_code != 200:
        # This means something went wrong.
        print("Error" , resp.status_code)

    return convert_response(resp)

### 
# wrapper for post
#
def post_response_as_json(base_url, data):
    post_headers, _ = update_header_token()
    resp = requests.post(base_url, headers=post_headers, data=data)
    if resp.status_code != 200:
        # This means something went wrong.
        print("Error" , resp.status_code)

    return convert_response(resp)

###
# Epoch time conversion
#
def get_localtime(t):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(t))


# create batch and 
def upload_athena_files(input_file, batch_file):
    session = boto3.Session()
    s3_client = session.client('s3')

    try:
        resp = s3_client.upload_file('build/'+input_file, my_bucket_name, my_prefix+'/'+input_file)
        resp = s3_client.upload_file('build/'+batch_file, my_bucket_name, my_prefix+'/'+batch_file)
    except ClientError as e:
        print(e)

### Inspect the Slurm REST API Schema

If you get a "Secrets Manager can't find the specified secret" error message, that means the cron job to store the slurm token in a secret has not been executed yet ( scheduled to run every 20 mins). 


In [ ]:
import requests
import json



slurm_openapi_ep = 'http://'+slurm_host+':8082/openapi/v3'
slurm_rest_base='http://'+slurm_host+':8082/slurm/v0.0.35'

_, get_headers = update_header_token()

resp_api = requests.get(slurm_openapi_ep, headers=get_headers)
print(resp_api)

if resp_api.status_code != 200:
    # This means something went wrong.
    print("Error" , resp_api.status_code)

with open('build/slurm_api.json', 'w') as outfile:
    json.dump(resp_api.json(), outfile)

print(json.dumps(resp_api.json(), indent=2))


### Use REST API callls to interact with ParallelCluster

Then we will make direct REST API requests to retrieve the partitions in response

If you get server errors, most likely
1. Cron job - token_refresher.sh (every 20 mins) hasn't been run yet after the IAM policy is updated. You can check for the slurm_token_yourClusterName secrete in AWS Secret Manager console. 
2. login to the head-node and check the system logs of "slurmrestd", which is running as a service. 


In [ ]:

partition_info = ["name", "nodes", "nodes_online", "total_cpus", "total_nodes"]

##### This works as well, 
# update header in case the token has expired
_, get_headers = update_header_token()

##### call REST API directly
slurm_partitions_url= slurm_rest_base+'/partitions/'
partitions = get_response_as_json(slurm_partitions_url)
#print(partitions['partitions'])
#20.02.4 returns a dict, not an array
print_table_from_dict(partition_info, partitions['partitions'])

# newer slurmrest return proper array
# print_table_from_json_array(partition_info, [partitions['partitions']['q1'], partitions['partitions']['q2']] )


### Submit a job
The slurm_rest_api_client job submit function doesn't include the "script" parameter. We will have to use the REST API Post directly. 

The body of the post should be like this.  

```
{"job": {"account": "test", "ntasks": 20, "name": "test18.1", "nodes": [2, 4],
"current_working_directory": "/tmp/", "environment": {"PATH": "/bin:/usr/bin/:/usr/local/bin/","LD_LIBRARY_PATH":
"/lib/:/lib64/:/usr/local/lib"} }, "script": "#!/bin/bash\necho it works"}
```
When the job is submitted through REST API, it will run as the user "slurm". That's what the work directory "/shared/tmp" should be owned by "slurm:slurm", which is done in the post_install script. 

fetch_and_run.sh will fetch the sbatch script and the input file from S3 and put them in /shared/tmp




### Program batch script, input and output files

To share the pcluster among different users and make sure users can only access their own input and output files, we will use user's ow S3 buckets for input and output files.

The job will be running on the ParallelCluster under /efs/tmp (for example) through a fatch (from the S3 bucket) and run script and the output will be stored in the same bucket under "output" path. 

If the simulation results are stored in vtk files, which are merged into single block vtk files from individual mesh block vtk files. The merging process is programmed in the batch script after the simulation executions. 

```
list=$(ls | grep block |grep vtk)

MAXBLOCK_ID=0
MAX_OUTSTEP=0
for l in $list
do
  PROB_ID=$(echo $l |cut -d '.' -f 1)
  OUTPUT_ID=$(echo $l| cut -d '.' -f 3 |cut --complement -c 1-3)

  mb_id=$(echo $l| cut -d '.' -f 2 | cut --complement -c 1-5)
  MAXBLOCK_ID=$(( mb_id > MAXBLOCK_ID? mb_id: MAXBLOCK_ID))

  os_id=$(echo $l| cut -d '.' -f 4)
  os_id=$((10#$os_id))
  MAX_OUTSTEP=$(( os_id > MAX_OUTSTEP? os_id: MAX_OUTSTEP))
done

cp /shared/athena-public-version/vis/vtk/join_* .
gcc -o join_vtk++ join_vtk++.c
./join_all_vtk.sh $PROB_ID $OUTPUT_ID $MAXBLOCK_ID $MAX_OUTSTEP

# copy the output files to S3 , excluding the block files
aws s3 cp . s3://$BUCKET_NAME/$PREFIX/$OUTPUT_FOLDER/ --recursive --exclude "*.block*"
```

In this notebook, we will use hdf5 format for the output data


In [ ]:


# Where the batch script, input file, output files are uploaded to S3
job_name = "orszag-tang-lowres"
my_prefix = "athena/"+job_name

# template files for input and batch script
input_file_ini = "config/athinput_orszag_tang.ini"
batch_file_ini = "config/batch_athena_sh.ini"

# actual input and batch script files
input_file = "athinput_orszag_tang.input"
batch_file = "batch_athena.sh"
    
################## Begin ###################################
# Mesh/Meshblock parameters
# nx1,nx2,nx3 - number of zones in x,y,z
# mbx1, mbx2, mbx3 - meshblock size 
# nx1/mbx1 X nx2/mbx2 X nx3/mbx3 = number of meshblocks - this should be the number of cores you are running the simulation on 
# e.g. mesh 100 X 100 X 100 with meshsize 50 X 50 X 50 will yield 2X2X2 = 8 blocks, run this on a cluster with 8 cores 
# 

#Mesh - actual domain of the problem 
# 512X512X512 cells with 64x64x64 meshblock - will have 8X8X8 = 512 meshblocks - if running on 32 cores/node, will need 
# 512/32=16 nodes
nx1=256
nx2=256
nx3=256

#Meshblock - each meshblock size - not too big 
mbnx1=64
mbnx2=64
mbnx3=64

# for c5n.18xlarge without HyperThreading, the number of cores is 32 - change this accordingly. 
num_cores_per_node = 32
num_of_threads = 1

# fake account_name 
account_name = "12345" 
partition = "q1"

# turn on/off EFA support in the script
use_efa="YES"
################# END ####################################

#Make sure the mesh is divisible by meshblock size
# e.g. num_blocks = (512/64)*(512/64)*(512/64) = 8 x 8 x 8 = 512
num_blocks = (nx1/mbnx1)*(nx2/mbnx2)*(nx3/mbnx3)

###
# Batch file parameters
# num_nodes should be less than or equal to the max number of nodes in your cluster
# num_tasks_per_node should be less than or equal to the max number of nodes in your cluster 
# e.g. 512 meshblocks / 32 core/node * 1 core/meshblock = 16 nodes -  c5n.18xlarge
num_nodes = num_blocks/num_cores_per_node

num_tasks_per_node = num_blocks/num_nodes/num_of_threads
cpus_per_task = num_of_threads


#This is where the program is installed on the cluster
exe_path = "/shared/athena-public-version/bin/athena"
#This is where the program is going to run on the cluster
work_dir = '/shared/tmp/'+job_name
ph = { '${nx1}': str(nx1), 
       '${nx2}': str(nx2),
       '${nx3}': str(nx3),
       '${mbnx1}': str(mbnx1),
       '${mbnx2}': str(mbnx2),
       '${mbnx3}': str(mbnx3), 
       '${num_of_threads}' : str(num_of_threads)}
pcluster_athena.template_to_file(input_file_ini, 'build/'+input_file, ph)

ph = {'${nodes}': str(num_nodes),
      '${ntasks-per-node}': str(int(num_tasks_per_node)),
      '${cpus-per-task}': str(cpus_per_task),
      '${account}': account_name,
      '${partition}': partition,
      '${job-name}': job_name,
      '${EXE_PATH}': exe_path,
      '${WORK_DIR}': work_dir,
      '${input-file}': input_file,
      '${BUCKET_NAME}': my_bucket_name,
      '${PREFIX}': my_prefix,
      '${USE_EFA}': use_efa,
      '${OUTPUT_FOLDER}': "output/",
      '${NUM_OF_THREADS}' : str(num_of_threads)}
pcluster_athena.template_to_file(batch_file_ini, 'build/'+batch_file, ph)

# upload to S3 for use later
upload_athena_files(input_file, batch_file)

job_script = "#!/bin/bash\n/shared/tmp/fetch_and_run.sh {} {} {} {} {}".format(my_bucket_name, my_prefix, input_file, batch_file, job_name)


In [ ]:

slurm_job_submit_base=slurm_rest_base+'/job/submit'

job_script = "#!/bin/bash\n/shared/tmp/fetch_and_run.sh {} {} {} {} {}".format(my_bucket_name,my_prefix, input_file, batch_file, job_name)

#in order to use Slurm REST to submit jobs, you need to have the working directory permission set to nobody:nobody. in this case /efs/tmp
data = {'job':{ 'account': account_name, 'partition': partition, 'name': 'my-athena', 'current_working_directory':'/shared/tmp/', 'environment': {"PATH": "/bin:/usr/bin/:/usr/local/bin/:/opt/slurm/bin:/opt/amazon/openmpi/bin","LD_LIBRARY_PATH":
"/lib/:/lib64/:/usr/local/lib:/opt/slurm/lib:/opt/slurm/lib64"}}, 'script':job_script}

###
# This job submission will generate two jobs , the job_id returned in the response is for the bash job itself. the sbatch will be the job_id+1 run subsequently.
#
resp_job_submit = post_response_as_json(slurm_job_submit_base, data=json.dumps(data))


print(resp_job_submit)


### List recent jobs

In [ ]:
# get the list of all the jobs immediately after the previous step. This should return two running jobs. 
slurm_jobs_base=slurm_rest_base+'/jobs'

jobs = get_response_as_json(slurm_jobs_base)
# print(jobs)
jobs_headers = [ 'job_id', 'job_state', 'account', 'batch_host', 'nodes', 'cluster', 'partition', 'current_working_directory']

# newer version of slurm 
#print_table_from_json_array(jobs_headers, jobs['jobs'])
print_table_from_json_array(jobs_headers, jobs)
                   

## Accounting 
The Slurm account information is managed by slurmdbd process and stored in a data store( local file or a relational database). In our setup, we use an AWS RDS MySQL database, which has IAM authentication enabled and is running in the save VPC as the ParallelCluster. We can access the database from this notebook. 


In [ ]:
### get the root ca and install mysql.connector
# only need to do this once per kernel
#

!wget https://s3.amazonaws.com/rds-downloads/rds-ca-2019-root.pem
!pip install mysql.connector


#### Step 6. Connect to the mysql database and query the parallel_job_table 

In [ ]:
import mysql.connector
from mysql.connector.constants import ClientFlag
import sys
import boto3
import os




rds_secret = json.loads(pcluster_athena.get_slurm_dbd_rds_secret(rds_secret_name))

ENDPOINT= rds_secret['host']
PORT=rds_secret['port']
USER=rds_secret['username']
PASS=rds_secret['password']
DBNAME="slurm_acct_db"

# This is the name of the cluster - default to "parallelcluster" - in this excercise, we default the cluster name to "parallelcluster"
# If you are using sacctmgr to track accounting for multiple clusters, this could be set differently
CLUSTERNAME="parallelcluster"

config = {
    'user': USER,
    'password': PASS,  
    'host': ENDPOINT,
    'port': PORT,
    'database': DBNAME,
}

table_headers=['job_db_inx', 'account', 'cpus_req', 'job_name', 'id_job', 'nodelist', 'partition', 'time_submit', 'time_start', 'time_end', 'duration(s)', 'work_dir']
table = [table_headers]

try:
    conn =  mysql.connector.connect(**config)
    cur = conn.cursor()
    cur.execute("""SELECT job_db_inx, account, cpus_req, job_name, id_job, nodelist, 'partition', time_submit, time_start, time_end, work_dir from {}_job_table""".format(CLUSTERNAME))
    query_results = cur.fetchall()
except Exception as e:
    print("Database connection failed due to {}".format(e)) 
    raise

#job_table_header =[(0,'job_id_inx'), (3, 'account'), ()) 
for r in query_results:
    l = list(r)
    # add a duration before the last element
    l.append(l[10])
    #duration
    if l[8] ==0 or l[9]==0:
        l[10] = 0
    else:
        l[10] = (l[9] - l[8])

    l[7] = get_localtime(l[7])
    l[8] = get_localtime(l[8])
    l[9] = get_localtime(l[9])
    table.append(l)
    

display_table(table)

# Visualize Athena++ Simulation Results
In this notebook, we are going to use the python library comes with Athena++ to read and visualize the simulation results.

In the previous notebook, we saved the simulation results in s3://<bucketname>/athema/$job_name/output folder

Import the hdf python code that came with Athena++

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import clear_output
import h5py

#Do this once. clone the athena++ source code , and the hdf5 python package we need is under vis/python folder

if not os.path.isdir('athena-public-version'):
    !git clone https://github.com/PrincetonUniversity/athena-public-version
else:
    print("Athena++ code already cloned, skip")
    
sys.path.insert(0, 'athena-public-version/vis/python')
import athena_read

In [ ]:
data_folder=job_name+'/output'
output_folder = my_bucket_name+'/athena/'+data_folder

if not os.path.isdir(job_name):
    !mkdir -p $job_name
    !aws s3 cp s3://$output_folder/ ./$data_folder/ --recursive
else:
    print('project folder exists, skip')

### Display the hst data
History data shows the overs all parameter changes over time. The time interval can be different from that of the hdf5 files.

In OrszagTang simulations, the variables in the hst files are 'time', 'dt', 'mass', '1-mom', '2-mom', '3-mom', '1-KE', '2-KE', '3-KE', 'tot-E', '1-ME', '2-ME', '3-ME'

All the variables a

In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

hst = athena_read.hst(data_folder+'/OrszagTang.hst')

# cannot use this reliably because hst and hdf can have different number of time steps. In this case,we have the same number of steps
num_timesteps = len(hst['time'])

print(hst.keys())

plt.plot(hst['time'], hst['dt'])


## Reading HDF5 data files 

The hdf5 data files contain all variables inside all meshblocks. There are some merging and calculating work to be done before we can visualizing the result. Fortunately ,Athena++ vis/hdf package takes care of the hard part. 


In [ ]:
# Let's example the content of the hdf files

f = h5py.File(data_folder+'/OrszagTang.out2.00001.athdf', 'r')
# variable lists <KeysViewHDF5 ['B', 'Levels', 'LogicalLocations', 'prim', 'x1f', 'x1v', 'x2f', 'x2v', 'x3f', 'x3v']>
print(f.keys())

#<HDF5 dataset "B": shape (3, 512, 64, 64, 64), type "<f4"> 
print(f['prim'])

### Simulation result data 

Raw athdf data has the following keys
<KeysViewHDF5 ['B', 'Levels', 'LogicalLocations', 'prim', 'x1f', 'x1v', 'x2f', 'x2v', 'x3f', 'x3v']>

After athena_read.athdf() call, the result contains keys, which can be used as the field name
['Coordinates', 'DatasetNames', 'MaxLevel', 'MeshBlockSize', 'NumCycles', 'NumMeshBlocks', 'NumVariables', 'RootGridSize', 'RootGridX1', 'RootGridX2', 'RootGridX3', 'Time', 'VariableNames', 'x1f', 'x1v', 'x2f', 'x2v', 'x3f', 'x3v', 'rho', 'press', 'vel1', 'vel2', 'vel3', 'Bcc1', 'Bcc2', 'Bcc3']


In [ ]:
def process_athdf(filename, num_step):
    print("Processing ", filename)
    athdf = athena_read.athdf(filename)
    return athdf

# extract list of fields and take a slice in one dimension, dimension can be 'x', 'y', 'z'
def read_all_timestep (data_file_name_template, num_steps, field_names, slice_number, dimension):

    if not dimension in ['x', 'y', 'z']:
        print("dimension can only be 'x/y/z'")
        return
    
    # would ideally process all time steps together and store themn in memory. However, they are too big, will have to trade time for memory 
    result = {}
    for f in field_names:
        result[f] = list()
        
    for i in range(num_steps):
        fn = data_file_name_template.format(str(i).zfill(5))
        athdf = process_athdf(fn, i)
        for f in field_names:
            if dimension == 'x':
                result[f].append(athdf[f][slice_number,:,:])
            elif dimension == 'y':
                result[f].append(athdf[f][:, slice_number,:])
            else:
                result[f].append(athdf[f][:,:, slice_number])
                        
    return result

def animate_slice(data):
    plt.figure()
    for i in range(len(data)):
        plt.imshow(data[i])
        plt.title('Frame %d' % i)
        plt.show()
        plt.pause(0.2)
        clear_output(wait=True)




In [ ]:

data_file_name_template = data_folder+'/OrszagTang.out2.{}.athdf'

# this is time consuming, try do it once
data = read_all_timestep(data_file_name_template, num_timesteps, ['press', 'rho'], 1, 'x')



In [ ]:
# Cycle through the time steps and look at pressure
animate_slice(data['press'])

In [ ]:
# Now look at density
animate_slice(data['rho'])

# Don't forget to clean up

1. Delete the ParallelCluster
2. Delete the RDS
3. S3 bucket
4. Secrets used in this excercise

Deleting VPC is risky, I will leave it out for you to manually clean it up if you created a new VPC. 

In [ ]:
!pcluster delete $pcluster_name

In [ ]:
importlib.reload(pcluster_athena)
importlib.reload(workshop)

db_name = 'pclusterdb'
slurm_secret_name='slurm_token_myTestCluster'

pcluster_athena.cleanup_cluster(my_bucket_name, db_name, slurm_secret_name, rds_secret_name, pcluster_name)
